# 📰 Полный EDA: Новости и Тикеры

**Цель:** Провести глубокий анализ новостей и их связи с тикерами:
- Базовая статистика новостей
- Качество маппинга тикеров
- Пересечения новостей ↔ свечей
- Временной анализ
- Визуализации

**Данные:**
- `data/preprocessed_news/train_news_with_tickers.parquet` - новости с привязанными тикерами
- `data/raw/participants/train_candles.csv` - исторические свечи

## 📦 Импорты и настройки

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from pathlib import Path
import json

# Настройки визуализации
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11
sns.set_style('whitegrid')
%matplotlib inline

# Отключаем предупреждения
import warnings
warnings.filterwarnings('ignore')

print("✅ Настройки применены")

## 📂 Загрузка данных

In [ ]:
# Загружаем новости с тикерами
news_path = Path('data/preprocessed_news/train_news_with_tickers.parquet')
stats_path = Path('data/preprocessed_news/train_news_with_tickers_stats.json')
candles_path = Path('data/raw/participants/train_candles.csv')

news = pd.read_parquet(news_path)
candles = pd.read_csv(candles_path)

# Загружаем статистику
with open(stats_path, 'r') as f:
    stats = json.load(f)

# Парсим списки тикеров
news['matched_tickers'] = news['matched_tickers'].apply(eval)

print(f"✅ Загружено:")
print(f"   📰 Новости: {len(news):,} строк")
print(f"   📊 Свечи: {len(candles):,} строк")
print(f"\nКолонки новостей: {news.columns.tolist()}")

---
# 1️⃣ Базовая статистика новостей

In [ ]:
# Конвертируем даты
news['publish_date'] = pd.to_datetime(news['publish_date'])
candles['begin'] = pd.to_datetime(candles['begin'])

print("=" * 60)
print("📋 БАЗОВАЯ СТАТИСТИКА НОВОСТЕЙ")
print("=" * 60)

print(f"\n📊 Общая информация:")
print(f"   Всего новостей: {len(news):,}")
print(f"   Период: {news['publish_date'].min().date()} — {news['publish_date'].max().date()}")
print(f"   Длительность: {(news['publish_date'].max() - news['publish_date'].min()).days} дней")

print(f"\n📝 Качество данных:")
print(f"   Пропуски в title: {news['title'].isna().sum()}")
print(f"   Пропуски в publication: {news['publication'].isna().sum()}")
print(f"   Дубликаты: {news.duplicated(subset=['publish_date', 'title']).sum()}")

print(f"\n📏 Длина текстов:")
news['title_len'] = news['title'].str.len()
news['text_len'] = news['publication'].str.len()

print(f"   Заголовки:")
print(f"      Средняя: {news['title_len'].mean():.0f} символов")
print(f"      Медиана: {news['title_len'].median():.0f} символов")
print(f"      Мин/Макс: {news['title_len'].min():.0f} / {news['title_len'].max():.0f}")

print(f"\n   Тексты публикаций:")
print(f"      Средняя: {news['text_len'].mean():.0f} символов")
print(f"      Медиана: {news['text_len'].median():.0f} символов")
print(f"      Мин/Макс: {news['text_len'].min():.0f} / {news['text_len'].max():.0f}")

In [ ]:
# Распределение по времени
news['year'] = news['publish_date'].dt.year
news['month'] = news['publish_date'].dt.to_period('M')
news['date'] = news['publish_date'].dt.date

print("\n📅 Распределение по годам:")
yearly_counts = news['year'].value_counts().sort_index()
for year, count in yearly_counts.items():
    print(f"   {year}: {count:,} новостей")

print(f"\n📆 Статистика по дням:")
daily_counts = news.groupby('date').size()
print(f"   Среднее новостей/день: {daily_counts.mean():.1f}")
print(f"   Медиана новостей/день: {daily_counts.median():.1f}")
print(f"   Макс новостей/день: {daily_counts.max()}")
print(f"   Дней без новостей: {(daily_counts == 0).sum()}")

---
# 2️⃣ Анализ маппинга тикеров

In [ ]:
print("=" * 60)
print("🎯 АНАЛИЗ МАППИНГА ТИКЕРОВ")
print("=" * 60)

# Покрытие
news_with_tickers = news['has_ticker'].sum()
coverage_pct = news['has_ticker'].mean() * 100

print(f"\n📊 Покрытие маппинга:")
print(f"   Новостей с тикерами: {news_with_tickers:,} ({coverage_pct:.1f}%)")
print(f"   Новостей без тикеров: {len(news) - news_with_tickers:,} ({100-coverage_pct:.1f}%)")

# Статистика по тикерам из JSON
print(f"\n📈 Топ-10 тикеров по упоминаниям:")
ticker_counts = stats['ticker_counts']
sorted_tickers = sorted(ticker_counts.items(), key=lambda x: x[1], reverse=True)
for i, (ticker, count) in enumerate(sorted_tickers[:10], 1):
    print(f"   {i:2}. {ticker}: {count:,} упоминаний")

# Множественные упоминания
news['num_tickers'] = news['matched_tickers'].apply(len)
print(f"\n🔗 Множественные упоминания:")
print(f"   Новостей с 1 тикером: {(news['num_tickers'] == 1).sum():,}")
print(f"   Новостей с 2+ тикерами: {(news['num_tickers'] >= 2).sum():,}")
print(f"   Среднее тикеров/новость: {news[news['has_ticker']]['num_tickers'].mean():.2f}")
print(f"   Максимум тикеров в одной новости: {news['num_tickers'].max()}")

In [ ]:
# Распределение множественных упоминаний
multi_ticker_dist = news['num_tickers'].value_counts().sort_index()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Гистограмма количества тикеров на новость
multi_ticker_dist.plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Распределение: количество тикеров на новость', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Количество тикеров')
axes[0].set_ylabel('Количество новостей')
axes[0].grid(axis='y', alpha=0.3)

# Топ-15 тикеров по упоминаниям
top_tickers = sorted_tickers[:15]
tickers_df = pd.DataFrame(top_tickers, columns=['Ticker', 'Count'])
axes[1].barh(tickers_df['Ticker'], tickers_df['Count'], color='coral')
axes[1].set_title('Топ-15 тикеров по упоминаниям', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Количество упоминаний')
axes[1].invert_yaxis()
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

---
# 3️⃣ Пересечения новостей ↔ свечей

In [ ]:
print("=" * 60)
print("🔄 ПЕРЕСЕЧЕНИЯ НОВОСТЕЙ ↔ СВЕЧЕЙ")
print("=" * 60)

# Тикеры из новостей
all_news_tickers = set(ticker_counts.keys())

# Тикеры из свечей
all_candles_tickers = set(candles['ticker'].unique())

print(f"\n📊 Сравнение тикеров:")
print(f"   Уникальные тикеры в новостях: {len(all_news_tickers)}")
print(f"   Уникальные тикеры в свечах: {len(all_candles_tickers)}")

only_news = all_news_tickers - all_candles_tickers
only_candles = all_candles_tickers - all_news_tickers
common = all_news_tickers & all_candles_tickers

print(f"\n✅ Общие тикеры: {len(common)} / {len(all_candles_tickers)} = {len(common)/len(all_candles_tickers)*100:.1f}%")
if only_news:
    print(f"⚠️  Только в новостях: {only_news}")
if only_candles:
    print(f"⚠️  Только в свечах: {only_candles}")

# Временное покрытие
news_date_range = (news['publish_date'].min(), news['publish_date'].max())
candles_date_range = (candles['begin'].min(), candles['begin'].max())

print(f"\n📅 Временное покрытие:")
print(f"   Новости: {news_date_range[0].date()} — {news_date_range[1].date()}")
print(f"   Свечи:   {candles_date_range[0].date()} — {candles_date_range[1].date()}")

# Проверка лага (новости должны быть <= t-1)
print(f"\n⏱️  Проверка лага (новости ≤ t-1):")
news_dates = set(news['publish_date'].dt.date)
candles_dates = set(candles['begin'].dt.date)
print(f"   Уникальных дат в новостях: {len(news_dates)}")
print(f"   Уникальных дат в свечах: {len(candles_dates)}")
print(f"   Пересекающихся дат: {len(news_dates & candles_dates)}")

In [ ]:
# Покрытие новостями каждого тикера
print("\n📰 Покрытие новостями по тикерам:")
print("\nТикер | Новости | Свечи | Покрытие")
print("-" * 45)

coverage_data = []
for ticker in sorted(all_candles_tickers):
    news_count = ticker_counts.get(ticker, 0)
    candles_count = len(candles[candles['ticker'] == ticker])
    
    coverage_data.append({
        'Ticker': ticker,
        'News': news_count,
        'Candles': candles_count
    })
    
    print(f"{ticker:5} | {news_count:6,} | {candles_count:5,} | {'✅' if news_count > 0 else '❌'}")

coverage_df = pd.DataFrame(coverage_data)

---
# 4️⃣ Временной анализ

In [ ]:
print("=" * 60)
print("⏰ ВРЕМЕННОЙ АНАЛИЗ")
print("=" * 60)

# Частота новостей по месяцам
monthly_counts = news.groupby('month').size()

print(f"\n📊 Статистика по месяцам:")
print(f"   Среднее новостей/месяц: {monthly_counts.mean():.0f}")
print(f"   Медиана: {monthly_counts.median():.0f}")
print(f"   Мин/Макс: {monthly_counts.min()} / {monthly_counts.max()}")

# Месяц с наибольшей активностью
max_month = monthly_counts.idxmax()
max_count = monthly_counts.max()
print(f"\n📈 Пик активности: {max_month} ({max_count} новостей)")

# Месяц с наименьшей активностью
min_month = monthly_counts.idxmin()
min_count = monthly_counts.min()
print(f"📉 Минимум активности: {min_month} ({min_count} новостей)")

In [ ]:
# Визуализация временных рядов
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# График 1: Общая активность новостей
monthly_counts_df = monthly_counts.to_frame(name='count')
monthly_counts_df.index = monthly_counts_df.index.to_timestamp()
monthly_counts_df.plot(ax=axes[0], color='steelblue', linewidth=2, legend=False)
axes[0].set_title('Количество новостей по месяцам (2020-2025)', fontsize=13, fontweight='bold')
axes[0].set_xlabel('')
axes[0].set_ylabel('Количество новостей')
axes[0].grid(alpha=0.3)
axes[0].axhline(monthly_counts.mean(), color='red', linestyle='--', alpha=0.5, label='Среднее')
axes[0].legend()

# График 2: Топ-5 тикеров по месяцам
# Explode новости по тикерам для временного анализа
news_exploded = news.explode('matched_tickers')
news_exploded = news_exploded[news_exploded['matched_tickers'].notna()]
news_exploded['month_ts'] = news_exploded['publish_date'].dt.to_period('M').dt.to_timestamp()

top_5_tickers = [t[0] for t in sorted_tickers[:5]]
for ticker in top_5_tickers:
    ticker_news = news_exploded[news_exploded['matched_tickers'] == ticker]
    ticker_monthly = ticker_news.groupby('month_ts').size()
    ticker_monthly.plot(ax=axes[1], label=ticker, linewidth=2, alpha=0.8)

axes[1].set_title('Топ-5 тикеров: динамика упоминаний', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Дата')
axes[1].set_ylabel('Количество упоминаний')
axes[1].legend(loc='upper left')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

---
# 5️⃣ Визуализации: Co-occurrence и сети

In [ ]:
print("=" * 60)
print("🔗 CO-OCCURRENCE АНАЛИЗ")
print("=" * 60)

# Строим матрицу co-occurrence (какие тикеры упоминаются вместе)
tickers_list = sorted(all_candles_tickers)
cooccurrence_matrix = pd.DataFrame(0, index=tickers_list, columns=tickers_list)

# Заполняем матрицу
for tickers in news['matched_tickers']:
    if len(tickers) >= 2:
        for i, t1 in enumerate(tickers):
            for t2 in tickers[i+1:]:
                if t1 in tickers_list and t2 in tickers_list:
                    cooccurrence_matrix.loc[t1, t2] += 1
                    cooccurrence_matrix.loc[t2, t1] += 1

print(f"\n📊 Матрица co-occurrence построена ({len(tickers_list)} x {len(tickers_list)})")
print(f"   Всего совместных упоминаний: {cooccurrence_matrix.sum().sum() / 2:.0f}")

# Топ-10 пар тикеров
print(f"\n🔝 Топ-10 пар тикеров (совместные упоминания):")
pairs = []
for i in range(len(tickers_list)):
    for j in range(i+1, len(tickers_list)):
        count = cooccurrence_matrix.iloc[i, j]
        if count > 0:
            pairs.append((tickers_list[i], tickers_list[j], count))

pairs_sorted = sorted(pairs, key=lambda x: x[2], reverse=True)
for i, (t1, t2, count) in enumerate(pairs_sorted[:10], 1):
    print(f"   {i:2}. {t1} ↔ {t2}: {count} раз")

In [ ]:
# Тепловая карта co-occurrence
plt.figure(figsize=(12, 10))
sns.heatmap(
    cooccurrence_matrix, 
    annot=True, 
    fmt='g', 
    cmap='YlOrRd', 
    square=True,
    linewidths=0.5,
    cbar_kws={'label': 'Количество совместных упоминаний'}
)
plt.title('Тепловая карта: совместные упоминания тикеров', fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Тикер', fontsize=11)
plt.ylabel('Тикер', fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
# Сетевой граф (требует networkx)
try:
    import networkx as nx
    
    # Создаем граф
    G = nx.Graph()
    
    # Добавляем узлы (все тикеры)
    for ticker in tickers_list:
        G.add_node(ticker)
    
    # Добавляем ребра (только если >= 10 совместных упоминаний)
    threshold = 10
    for t1, t2, count in pairs_sorted:
        if count >= threshold:
            G.add_edge(t1, t2, weight=count)
    
    # Визуализация
    plt.figure(figsize=(14, 10))
    pos = nx.spring_layout(G, k=2, iterations=50, seed=42)
    
    # Размер узлов = количество упоминаний
    node_sizes = [ticker_counts.get(ticker, 0) for ticker in G.nodes()]
    
    # Рисуем
    nx.draw_networkx_nodes(
        G, pos, 
        node_size=[s/5 for s in node_sizes],  # масштабируем
        node_color='lightblue',
        edgecolors='navy',
        linewidths=2
    )
    
    # Ребра с толщиной = вес
    edges = G.edges()
    weights = [G[u][v]['weight'] for u, v in edges]
    nx.draw_networkx_edges(
        G, pos, 
        width=[w/30 for w in weights],
        alpha=0.5,
        edge_color='gray'
    )
    
    # Подписи
    nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')
    
    plt.title(f'Сетевой граф: связи между тикерами (>= {threshold} упоминаний)', 
              fontsize=14, fontweight='bold', pad=20)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print(f"✅ Сеть построена: {G.number_of_nodes()} узлов, {G.number_of_edges()} ребер")
    
except ImportError:
    print("⚠️  networkx не установлен. Для визуализации графа выполните: pip install networkx")

---
# 6️⃣ Примеры новостей

In [ ]:
print("=" * 60)
print("📰 ПРИМЕРЫ НОВОСТЕЙ")
print("=" * 60)

# 3 случайные новости с найденными тикерами
print("\n✅ Примеры с успешным маппингом:\n")
successful = news[news['has_ticker']].sample(3, random_state=42)
for idx, row in successful.iterrows():
    print(f"📅 {row['publish_date'].date()}")
    print(f"🎯 Тикеры: {', '.join(row['matched_tickers'])}")
    print(f"📰 {row['title']}")
    print(f"📝 {row['publication'][:200]}...")
    print("-" * 60)

# 3 новости без тикеров
if (~news['has_ticker']).sum() > 0:
    print("\n❌ Примеры без маппинга:\n")
    failed = news[~news['has_ticker']].sample(min(3, (~news['has_ticker']).sum()), random_state=42)
    for idx, row in failed.iterrows():
        print(f"📅 {row['publish_date'].date()}")
        print(f"📰 {row['title']}")
        print(f"📝 {row['publication'][:200]}...")
        print("-" * 60)

---
# 📊 Итоговая сводка

In [ ]:
print("=" * 60)
print("📋 ИТОГОВАЯ СВОДКА")
print("=" * 60)

print(f"\n✅ КАЧЕСТВО ДАННЫХ:")
print(f"   Новостей: {len(news):,}")
print(f"   Покрытие маппинга: {coverage_pct:.1f}%")
print(f"   Тикеров: {len(all_candles_tickers)}")
print(f"   Совпадение тикеров новости↔свечи: {len(common)}/{len(all_candles_tickers)} (100%)")

print(f"\n📊 РАСПРЕДЕЛЕНИЕ:")
print(f"   Среднее упоминаний/тикер: {sum(ticker_counts.values()) / len(ticker_counts):.0f}")
print(f"   Топ тикер: {sorted_tickers[0][0]} ({sorted_tickers[0][1]:,} упоминаний)")
print(f"   Среднее новостей/день: {daily_counts.mean():.1f}")

print(f"\n🎯 РЕКОМЕНДАЦИИ:")
if coverage_pct < 90:
    print(f"   ⚠️  Покрытие < 90% - рассмотреть улучшение алгоритма маппинга")
else:
    print(f"   ✅ Покрытие >= 90% - маппинг работает хорошо")

if len(only_candles) > 0:
    print(f"   ⚠️  Есть тикеры без новостей: {only_candles}")
else:
    print(f"   ✅ Все тикеры из свечей покрыты новостями")

print(f"\n✅ Анализ завершен!")